# Lesson-02

1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

-Наименование вакансии

-Предлагаемую зарплату (отдельно мин. отдельно макс. и отдельно валюту)

-Ссылку на саму вакансию


*Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [34]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [35]:
def hh_parser(vac):
    
    link = 'https://hh.ru/search/vacancy'
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
        }
    
    df = pd.DataFrame(columns =('Название', 'Ссылка', 'Мин. з/п', 'Макс. з/п', 'Валюта', 'Сайт'))
    
    
    for i in range(100):
        params = {
            'st': 'searchVacancy', \
            'text': vac, \
            'search_field': 'name', \
            'search_period': '7', \
            'items_on_page': '50', \
            'page': i
        }

        html = requests.get(link, params=params, headers=headers)

        if html.ok:
            soup = bs(html.text,'lxml')
            vacancy1 = soup.find('div', {'class':'vacancy-serp-item'})
            div_on_page = len(soup.find('div', {'class':"vacancy-serp"}))
            vacancy_site_link = 'https://hh.ru/'

            for i in range(div_on_page):
                try:
                    ### Название вакансии
                    vacancy_name = vacancy1.find('span', {'class':'g-user-content'}).text

                    ### Ссылка на вакансию
                    vacancy_link = vacancy1.find('span', {'class':'g-user-content'})
                    vacancy_link = vacancy_link.next['href']

                    ### З/п на вакансию
                    vacancy_salary = vacancy1.find('div', {'class':'vacancy-serp-item__sidebar'}).next.text
                    vacancy_salary = vacancy_salary.replace('\xa0', '')
                    vacancy_salary = vacancy_salary.replace(' ', '-')
                    vacancy_salary = vacancy_salary.split('-')
                    vacancy_salary_min = vacancy_salary[0]
                    vacancy_salary_max = vacancy_salary[1]
                    vacancy_salary_currency = vacancy_salary[2]


                    try:
                        vacancy_salary_min = int(vacancy_salary_min)
                    except:
                        vacancy_salary_min = float('Nan')

                    try:
                        vacancy_salary_max = int(vacancy_salary_max)
                    except:
                        vacancy_salary_max = float('Nan')

                    if vacancy_salary_currency == ('руб.'):
                        vacancy_salary_currency = 'RUB'
                    elif vacancy_salary_currency == ('EUR'):
                        vacancy_salary_currency = 'EUR'
                    elif vacancy_salary_currency == ('USD'):
                        vacancy_salary_currency = 'USD'
                    else:
                        vacancy_salary_currency = float('Nan')

                    df = df.append(
                        {
                        'Название': vacancy_name,
                        'Ссылка': vacancy_link,
                        'Мин. з/п': vacancy_salary_min,
                        'Макс. з/п': vacancy_salary_max,
                        'Валюта': vacancy_salary_currency,
                        'Сайт': vacancy_site_link,
                        },
                        ignore_index=True,
                        )
                    vacancy1 = vacancy1.next_sibling
                except:
                    vacancy1 = vacancy1.next_sibling
        if soup.find('a', {'class':'HH-Pager-Controls-Next'}) == None:
            break
    return df

In [36]:
def superjob_parser(vac):
    link = 'https://russia.superjob.ru'
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
        }
    df = pd.DataFrame(columns =('Название', 'Ссылка', 'Мин. з/п', 'Макс. з/п', 'Валюта', 'Сайт'))

    final_link = link + '/vacancy/search/'

    for k in range(30):
            params = {
                'keywords': vac, \
                'page': k, \
                }

            html = requests.get(final_link, params=params, headers=headers)

            if html.ok:
                soup = bs(html.text,'lxml')
                vacancy1 = soup.find('div', {'class':'f-test-vacancy-item'}).parent.parent
                vacancy_on_page = vacancy1.findChildren('div', {'class':'iJCa5'})


                for i in vacancy_on_page:
                    vacancy_name = i.find_next('div', {'class':'CuJz5'}).text

                    vacancy_link = i.find_next('div', {'class':'CuJz5'})
                    vacancy_salary = vacancy_link.next_sibling
                    vacancy_link = vacancy_link.next['href']
                    vacancy_link = link + vacancy_link
                    vacancy_link

                    vacancy_salary = vacancy_salary.text
                    vacancy_salary = vacancy_salary.split('\xa0')
                    
                    if vacancy_salary[0] == 'до':
                        vacancy_salary = [float('Nan'), vacancy_salary[1]+vacancy_salary[2], vacancy_salary[3]]
                    elif vacancy_salary[0] == 'от':
                        vacancy_salary = [vacancy_salary[1]+vacancy_salary[2], float('Nan'), vacancy_salary[3]]
                    elif len(vacancy_salary) == 1:
                        vacancy_salary = [float('Nan'), float('Nan'), float('Nan')]
                    elif vacancy_salary[2] == '-':
                        vacancy_salary = [vacancy_salary[0]+vacancy_salary[1], vacancy_salary[3]+vacancy_salary[4], vacancy_salary[5]]
                    elif len(vacancy_salary) == 3:
                        pass
                    else:
                        vacancy_salary = [float('Nan'), float('Nan'), float('Nan')]
                    
                    if vacancy_salary[2] == ('руб.'):
                        vacancy_salary[2] = 'RUB'
                    elif vacancy_salary[2] == ('EUR'):
                        vacancy_salary[2] = 'EUR'
                    elif vacancy_salary[2] == ('USD'):
                        vacancy_salary[2] = 'USD'
                    else:
                        vacancy_salary[2] = float('Nan')
                    
                    df = df.append(
                        {
                        'Название': vacancy_name,
                        'Ссылка': vacancy_link,
                        'Мин. з/п': vacancy_salary[0],
                        'Макс. з/п': vacancy_salary[1],
                        'Валюта': vacancy_salary[2],
                        'Сайт': link,
                        },
                        ignore_index=True,
                        )


            if soup.find(text='Дальше') == None:
                break
    return df

In [46]:
def hh_superjob_parser(keyword):
    keyword = str(keyword)
    df1 = hh_parser(keyword)
    df2 = superjob_parser(keyword)
    df = pd.concat([df1, df2], axis=0)
    return df

In [50]:
hh_superjob_parser('Python')

,Название,Ссылка,Мин. з/п,Макс. з/п,Валюта,Сайт
0,Программист Python,https://spb.hh.ru/vacancy/36615032?query=Python,170000,250000,RUB,https://hh.ru/
1,Python Developer,https://spb.hh.ru/vacancy/35095020?query=Python,NaN,2500,USD,https://hh.ru/
2,Python Backend Developer (Munchen),https://spb.hh.ru/vacancy/36414126?query=Python,4200,6250,EUR,https://hh.ru/
3,Backend python/Django разработчик,https://spb.hh.ru/vacancy/36622564?query=Python,120000,200000,RUB,https://hh.ru/
4,Backend Python разработчик,https://spb.hh.ru/vacancy/36563904?query=Python,120000,150000,RUB,https://hh.ru/
...,...,...,...,...,...,...
68,Главный инженер в отдел баз данных MSSQL и Pos...,https://russia.superjob.ru/vakansii/glavnyj-in...,NaN,NaN,NaN,https://russia.superjob.ru
69,Ведущий инженер в отдел серверных систем Linux,https://russia.superjob.ru/vakansii/veduschij-...,NaN,NaN,NaN,https://russia.superjob.ru
70,Инженер в отдел баз данных Hadoop и noSQL,https://russia.superjob.ru/vakansii/inzhener-v...,NaN,NaN,NaN,https://russia.superjob.ru
71,Инженер (программирование),https://russia.superjob.ru/vakansii/inzhener-3...,40000,NaN,RUB,https://russia.superjob.ru
